# Esercizio 10

[MAFFT](https://www.ebi.ac.uk/Tools/msa/mafft/) è un tool di allineamento multiplo sviluppato da EMBL-EBI (European Bioinformatics Institute - European Molecular Biology Laboratory) per sequenze di DNA.

Usare MAFFT (scegliendo ClustalW come formato di output) per allineare i 14 genomi di SARS-CoV-2 presenti nel file `covid-sequences.fasta` sequenziati nel novembre 2021 e scaricati dal sito di [NCBI](https://www.ncbi.nlm.nih.gov/sars-cov-2/). Il primo, con identificatore `NC_045512.2`, è il genoma di riferimento.

Trovare in seguito tutte le variazioni rispetto ai genoma di riferimento.

---

**Variazione**: una colonna nell'allineamento in cui esiste almeno una sequenza che ha *mismatch* con il riferimento.

Esempio di allineamento multiplo tra tre genomi `G_REF` (*reference*) `G1` e `G2`, che ha quattro variazioni nelle colonne 5, 8, 13 e 16:

    G_REF   AAGCTGATTGCACGC-T
    G1      --GCAGAGTGCAGGCCT
    G2      --GCCGAGTGCACGCCT

**Variazione 5**: `T` nel reference e `A` in G1 e `C` in G2.

**Variazione 8**: `T` nel reference e `G` sia in G1 e G2.

**Variazione 13**: `C` nel reference e `G` in G1.

**Variazione 16**: `-` (cancellazione) nel reference e `C` sia in G1 che in G2.

---

Si chiede di:
- costruire il *data frame* delle variazioni in cui le colonne del *data frame* sono le colonne di variazione nell'allineamento multiplo e le righe sono indicizzate con l'identificatore del genoma. Non considerare le colonne che cadono nei gap iniziali e finali.
- estrarre il genoma con più variazioni e quello con meno variazioni (rispetto al *reference*)
- ottenere il *data frame* delle variazioni "complete", cioè in cui tutti i genomi variano rispetto al *reference*.
- produrre il *data frame* delle variazioni "stabili", cioé in cui tutti i genomi variano allo stesso modo rispetto al riferimento (hanno la stessa base). 
- ottenere la lista delle colonne in cui c'è un gap nel genoma di riferimento.
- ottenere la lista delle colonne in cui c'è un gap in almeno uno dei genomi (che non siano il *reference*)

Importare Biopython.

In [27]:
import Bio

Importare il package `AlignIO` che è il package per manipolare file contenenti allineamenti multipli in diversi formati (tra cui `clustal`, formato del file in input).

In [28]:
from Bio import AlignIO

In [29]:
help(AlignIO)

Help on package Bio.AlignIO in Bio:

NAME
    Bio.AlignIO - Multiple sequence alignment input/output as alignment objects.

DESCRIPTION
    The Bio.AlignIO interface is deliberately very similar to Bio.SeqIO, and in
    fact the two are connected internally.  Both modules use the same set of file
    format names (lower case strings).  From the user's perspective, you can read
    in a PHYLIP file containing one or more alignments using Bio.AlignIO, or you
    can read in the sequences within these alignments using Bio.SeqIO.
    
    Bio.AlignIO is also documented at http://biopython.org/wiki/AlignIO and by
    a whole chapter in our tutorial:
    
    * `HTML Tutorial`_
    * `PDF Tutorial`_
    
    .. _`HTML Tutorial`: http://biopython.org/DIST/docs/tutorial/Tutorial.html
    .. _`PDF Tutorial`: http://biopython.org/DIST/docs/tutorial/Tutorial.pdf
    
    Input
    -----
    For the typical special case when your file or handle contains one and only
    one alignment, use the func

## Leggere l'allineamento in input

Il package `AlignIO` mette a disposizione la funzione `read` per leggere un file contenente un allineamento:

       AlignIO.read(input_file_name, format)
       
e restituisce un oggetto `MultipleSeqAlignment` che è un oggetto iterabile contenente oggetti `SeqRecord`, un oggetto per ognuna delle righe dell'allineamento letto.

In [30]:
alignment = AlignIO.read('./files/mafft-alignments-early.clustalw', 'clustal')

In [31]:
alignment

<<class 'Bio.Align.MultipleSeqAlignment'> instance (12 records of length 29903) at 11190e880>

Ottenere la lunghezza dell'allineamento letto, intesa come numero di colonne della matrice di allineamento, tramite il metodo `get_alignment_length()` dell'oggetto `MultipleSeqAlignment`.

In [32]:
alignment.get_alignment_length()

29903

Trasformare l'allineamento in una lista di oggetti `SeqRecord`.

In [33]:
alignment = list(alignment)

In [34]:
alignment

[SeqRecord(seq=Seq('ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGATCTCTTGT...AAA'), id='NC_045512.2', name='<unknown name>', description='NC_045512.2', dbxrefs=[]),
 SeqRecord(seq=Seq('NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...NNN'), id='HG998648.1', name='<unknown name>', description='HG998648.1', dbxrefs=[]),
 SeqRecord(seq=Seq('------------------------------------------------------...---'), id='MW751133.1', name='<unknown name>', description='MW751133.1', dbxrefs=[]),
 SeqRecord(seq=Seq('------------------------------------------------------...---'), id='MW751146.1', name='<unknown name>', description='MW751146.1', dbxrefs=[]),
 SeqRecord(seq=Seq('----------------------------AAACCAACCAACTTTCGATCTCTTGT...---'), id='MT020782.1', name='<unknown name>', description='MT020782.1', dbxrefs=[]),
 SeqRecord(seq=Seq('ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGATCTCTTGT...AAA'), id='MW422000.1', name='<unknown name>', description='MW422000.1', dbxrefs=[]),
 SeqRecord(seq=Seq('

## Eliminare dall'allineamento i gap iniziali.

Trovare il più lungo prefisso di soli simboli `-` presente nelle righe dell'allineamento. Supponendo che tale prefisso sia lungo `g`, eliminare da ogni riga dell'allineamento il prefisso di lunghezza `g`.

Ad esempio il seguente allineamento composto da tre righe:

    GTATGTGTCATGTTTTTGCTA
    --ATGTGTCATG-TTT-----
    ----GTGTCATGTTTTTG---
    
presenta un più lungo prefisso di soli simboli `-` di lunghezza `g=4` (terza riga). Eliminando da tutte le righe un prefisso di lunghezza 4 si ottiene:

        GTGTCATGTTTTTGCTA
        GTGTCATG-TTT-----
        GTGTCATGTTTTTG---

a) Determinare la lista delle lunghezze dei gap.

In [35]:
import re

In [36]:
gap_size_list = [len(re.findall(r'^-*', str(row.seq)).pop(0)) for row in alignment]

In [37]:
gap_size_list

[0, 0, 54, 54, 28, 0, 6, 6, 42, 54, 54, 0]

b) Estrarre la massima lunghezza dei gap iniziali.

In [38]:
max_leading_gap_length = max(gap_size_list)

In [39]:
max_leading_gap_length

54

c) Rimuovere dalle righe dell'allineamento un prefisso di lunghezza pari a quella massima trovata.

In [40]:
alignment = [row[max_leading_gap_length:] for row in alignment]

In [41]:
alignment

[SeqRecord(seq=Seq('AGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTCGGCTGCAT...AAA'), id='NC_045512.2', name='<unknown name>', description='NC_045512.2', dbxrefs=[]),
 SeqRecord(seq=Seq('AGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTCGGCTGCAT...NNN'), id='HG998648.1', name='<unknown name>', description='HG998648.1', dbxrefs=[]),
 SeqRecord(seq=Seq('AGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTCGGCTGCAT...---'), id='MW751133.1', name='<unknown name>', description='MW751133.1', dbxrefs=[]),
 SeqRecord(seq=Seq('AGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTCGGCTGCAT...---'), id='MW751146.1', name='<unknown name>', description='MW751146.1', dbxrefs=[]),
 SeqRecord(seq=Seq('AGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTCGGCTGCAT...---'), id='MT020782.1', name='<unknown name>', description='MT020782.1', dbxrefs=[]),
 SeqRecord(seq=Seq('AGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTCGGCTGCAT...AAA'), id='MW422000.1', name='<unknown name>', description='MW422000.1', dbxrefs=[]),
 SeqRecord(seq=Seq('

## Eliminare dall'allineamento i gap finali.

Trovare il più lungo suffisso di soli simboli `-` presente nelle righe dell'allineamento. Supponendo che tale suffisso sia lungo `g`, eliminare da ogni riga dell'allineamento il suffisso di lunghezza `g`.

Ad esempio il seguente allineamento composto da tre righe:

        GTGTCATGTTTTTGCTA
        GTGTCATG-TTT-----
        GTGTCATGTTTTTG---
        
presenta un più lungo suffisso di soli simboli `-` di lunghezza `g=5` (seconda riga). Eliminando da tutte le righe un suffisso di lunghezza 5 si ottiene:

        GTGTCATGTTTT
        GTGTCATG-TTT
        GTGTCATGTTTT

a) Determinare la lista delle lunghezze dei gap.

In [42]:
gap_size_list = [len(re.findall(r'-*$', str(row.seq)).pop(0)) for row in alignment]

In [43]:
gap_size_list

[0, 0, 67, 67, 57, 0, 58, 43, 63, 135, 135, 0]

b) Estrarre la massima lunghezza dei gap.

In [44]:
max_trailing_gap_length = max(gap_size_list)

In [45]:
max_trailing_gap_length

135

c) Rimuovere dalle righe dell'allineamento un suffisso di lunghezza pari a quella massima trovata.

In [46]:
alignment = [row[:-max_trailing_gap_length] for row in alignment]

In [47]:
alignment

[SeqRecord(seq=Seq('AGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTCGGCTGCAT...GAA'), id='NC_045512.2', name='<unknown name>', description='NC_045512.2', dbxrefs=[]),
 SeqRecord(seq=Seq('AGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTCGGCTGCAT...GAA'), id='HG998648.1', name='<unknown name>', description='HG998648.1', dbxrefs=[]),
 SeqRecord(seq=Seq('AGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTCGGCTGCAT...GAA'), id='MW751133.1', name='<unknown name>', description='MW751133.1', dbxrefs=[]),
 SeqRecord(seq=Seq('AGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTCGGCTGCAT...GAA'), id='MW751146.1', name='<unknown name>', description='MW751146.1', dbxrefs=[]),
 SeqRecord(seq=Seq('AGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTCGGCTGCAT...GAA'), id='MT020782.1', name='<unknown name>', description='MT020782.1', dbxrefs=[]),
 SeqRecord(seq=Seq('AGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTCGGCTGCAT...GAA'), id='MW422000.1', name='<unknown name>', description='MW422000.1', dbxrefs=[]),
 SeqRecord(seq=Seq('

## Creare il *data frame* delle variazioni

a) Creare il seguente dizionario

- `key`: posizione (1-based) della colonna di variazione all'interno dell'allineamento

- `value`: lista delle basi coinvolte nella variazione (il primo elemento si riferisce al *reference*). Se un genoma non presenta differenza rispetto al *reference*, deve essere  inserita la stringa vuota.

Non tenere conto di basi ambigue.

In [48]:
df_variant = {}

Estrarre la riga del *reference* (e rimuoverla dall'allineamento).

In [49]:
reference = alignment.pop(0)

In [50]:
reference

SeqRecord(seq=Seq('AGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTCGGCTGCAT...GAA'), id='NC_045512.2', name='<unknown name>', description='NC_045512.2', dbxrefs=[])

In [51]:
for (i,c) in enumerate(reference):
    variant_list = [(j, row[i]) for (j, row) in enumerate(alignment) if row[i] != c and row[i] in ['A', 'C', 'G', 'T', '-']]
    if variant_list != []:
        df_variant_list = [variant[1] for variant in variant_list]
        
        all_index_set = set(range(len(alignment)))
        variant_index_set = set(variant[0] for variant in variant_list)
        
        non_variant_index_set = all_index_set.difference(variant_index_set)
        
        for index in non_variant_index_set:
            df_variant_list.insert(index, '')
            
        df_variant_list.insert(0, c)
        
        df_variant[i+max_leading_gap_length+1] = df_variant_list

In [52]:
df_variant

{361: ['A', '', '', '', '', '', '', '', '', '', '', 'G'],
 490: ['T', '', '', '', '', '', '', '', 'A', '', '', ''],
 888: ['C', '', '', '', '', '', '', '', '', '', '', 'T'],
 922: ['G', 'A', '', '', '', '', '', '', '', '', '', ''],
 2102: ['C', '', '', '', '', '', '', '', '', '', '', 'T'],
 2447: ['G', '', '', '', '', '', '', '', '', '', '', 'T'],
 2461: ['T', '', '', '', '', 'C', '', '', '', '', '', ''],
 3177: ['C', '', '', '', '', '', '', '', 'T', '', '', ''],
 3638: ['G', '', '', '', '', 'T', '', '', '', '', '', ''],
 3990: ['C', 'T', '', '', '', '', '', '', '', '', '', ''],
 4402: ['T', '', '', '', 'C', '', '', '', '', '', '', ''],
 4551: ['C', '', '', '', '', '', '', '', '', '', '', 'T'],
 5044: ['G', '', '', '', '', '', '', '', '', '', '', 'A'],
 5062: ['G', '', '', '', 'T', '', '', '', '', '', '', ''],
 5833: ['C', '', '', '', '', '', '', '', 'T', '', '', ''],
 6364: ['G', '', '', '', '', '', '', '', '', '', '', 'A'],
 6539: ['C', '', '', '', '', '', '', '', '', 'T', 'T', ''],


b) Determinare la lista degli identificatori dei genomi da usare come indici (chiavi primarie).

In [32]:
index_list

['NC_045512.2',
 'HG998648.1',
 'MW751133.1',
 'MW751146.1',
 'MT020782.1',
 'MW422000.1',
 'MT730116.1',
 'MT730117.1',
 'MW593327.1',
 'MW635193.1',
 'MW635200.1',
 'MW598425.1']

c) Creare il data frame

    df = pd.DataFrame(df_data, index = index_list)

## Estrarre il genoma con più variazioni e quello con meno variazioni

a) Determinare la lista del numero di variazioni per genoma rispetto al riferimento.

b) Estrarre il genoma con più variazioni.

'MW598425.1'

c) Estrarre il genoma con meno variazioni.

'MT020782.1'

## Estrarre il *data frame* delle variazioni "complete"

In [42]:
df_complete

,8782
NC_045512.2,C
HG998648.1,T
MW751133.1,T
MW751146.1,T
MT020782.1,T
MW422000.1,T
MT730116.1,T
MT730117.1,T
MW593327.1,T
MW635193.1,T


## Estrarre il *data frame* delle variazioni "stabili"

In [44]:
df_stable

,8782
NC_045512.2,C
HG998648.1,T
MW751133.1,T
MW751146.1,T
MT020782.1,T
MW422000.1,T
MT730116.1,T
MT730117.1,T
MW593327.1,T
MW635193.1,T


## Estrarre la lista delle colonne in cui c'è un gap nel *reference*.

In [46]:
ref_gaps

[]

## Estrarre la lista delle colonne in cui c'è un gap in almeno uno dei genomi (diversi dal *reference*).

In [48]:
other_gaps

[26158, 26159, 26160, 26161]